# Running MultiPsi, input/code structure

There are two main ways to run MultiPsi. The first one is using a traditional way using an input file, and the second one is as a python script. For advanced users familiar with python programming, running a python script is the most flexible and powerful option while the input format is more adapted for new users as it is simple and has more failsafes.

## Running with an input file

To run multipsi, you can simply run in a terminal

`multipsi file.inp`

which will print the output to the screen, or redirect the input

`multipsi file.inp file.out`

You can find example inputs in the `examples` directory. The general input structure is composed of blocks delimited by `@`. There is one block for each module (e.g. mcscf, ci, response, etc...) as well as a couple of blocks to define the molecule, active space and method (resp. `@molecule`, `@orbitals` and `@method settings`). The order of the blocks is irrelevant. Within each block are a number of options written as `keyword:value`, and the value can be multiple lines.

The lines before the first block in the input are called "keylines" and can be used to provide a short-hand input which can be enough for most basic calculations. Thus, a complete input to run a CASSCF with an active space containing 12 electrons in 9 orbitals (full valence) with the def2-svp basis set, followed by a linear response calculation (time-dependent or TD) of the first 5 excited states is simply:

```
casscf(12,9) def2-svp td:5 # This is the keyline

@molecule
xyz:
O 0.0000   0.0000   0.0000
O 0.0000   1.0885   0.6697
O 0.0000  -1.0885   0.6697
@end
```

which is fully identical to the long-form input:

```
@molecule
xyz:
O 0.0000   0.0000   0.0000
O 0.0000   1.0885   0.6697
O 0.0000  -1.0885   0.6697
@end

@orbitals
cas: 12, 9
@end

@method settings
basis: def2-svp
wavefunction: mcscf
@end

@response
n_states:5
@end
```

## Running with a python script

MultiPsi is a python module and can thus be run natively as a python script. For this, you first need to import multipsi and veloxchem:

In [1]:
import veloxchem as vlx
import multipsi as mtp

Then the input above can be run by the following script:

In [6]:
# Define molecule and basis
mol_str = """
O 0.0000   0.0000   0.0000
O 0.0000   1.0885   0.6697
O 0.0000  -1.0885   0.6697
"""

molecule = vlx.Molecule.read_str(mol_str)
molecule.set_charge(0) # default
molecule.set_multiplicity(1) # default

basis = vlx.MolecularBasis.read(molecule,"def2-svp")

# Generate guess
orbguess = mtp.OrbitalGuess()
guessorb = orbguess.compute(molecule, basis)

# Define the active space (full valence = 9 2p orbitals, 12 electrons)
orb_space = mtp.OrbSpace(molecule, guessorb)
orb_space.cas(12, 9)

# MCSCF energy
mcscf_drv = mtp.McscfDriver()
mcscf_dict = mcscf_drv.compute(molecule, basis, orb_space)

# Linear response calculation
mcrpa = mtp.Mclr_EigenSolver()
mcrpa.max_vector_per_equation = 15
response_dict = mcrpa.compute(molecule, basis, mcscf_drv, n_states = 5) 

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 6
               Number of active orbitals:              9
               Number of virtual orbitals:             27

               This is a CASSCF wavefunction: CAS(12,9)

               CI expansion:
               -------------
               Number of determinants:      7056


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────────

The script version also provides useful tools to visualize the results, check the orbitals, etc...

You can also use the built-in help functions of python to know the available functions and keywords of modules:

In [ ]:
help(mtp.McscfDriver)